### Creation of the 19k ortholog alignment and phylogeny for Parkinson 2018
This notebook will document the work that was necessary to go from the following output files:
* Filtered assembly .fasta files (one per species)
* Predicted ORF files .fasta files (one per species)
* Predicted one-to-one ortholog file (one file for all four species)

To an amino acid alignment made through the concatenation of all 19000 ortholog peptide sequences and a ML phylogeny.

It will also document the dN/dS analysis conducted as part of this study.

This documentaion will be split up into three major sections
* #### [Fixing multi-ORF prediction](#fixing_multi_orf_prediction)
* #### [Creating the super alignment and phylogeny](#creating_the_super_alignment_and_phylogeny)
* #### [Creating the guidance alignments (mafft) and dropping low confidence columns](#conducting_alignments_in_guidance)
* #### [Creating phylip formated alignment blocks and running CODEML](#making_block_alignments_for_CODEML_analysis)
* #### [Creating BUSTED input files and running analyses](#creating_busted_input_files_and_running_analysis)
<hr />

<a id='fixing_multi_orf_prediction'></a>
### Fixing multi-ORF prediction
When we were working through the predicted one-to-one ortholog file e.g. /Users/humebc/Google Drive/projects/parky/protein_alignments/a_id.csv we were finding that some of the sets of amino acid sequences (four sequences, one per species) did not align well. Looking back into why this could be we found that the problem lay in the fact that although the post-filtering transcript sequences e.g. compXXX_seqXXX were unique per species assembly file, several ORFs could be predicted per transcript. When the one-to-one data was created, rather than referencing the ORF ID e.g. m.XXX (which is unique per species) the compXXX ID was used. As such, for those transcripts that had several ORFs predicted for them, it was pure chance that the correct ORF (that had been found to be an ortholog of one of the other species ORFs) had been selected.

The easiest way to solve this problem would have been to go back to the original input and output of the one-to-one ortholog predictions and work with the unique ORF IDs (e.g. m.XXX). However, this analysis was done several years ago and I was unable to find this file. As such, a different approach was required. To fix the issue I decided to go through each of the ortholog predictions and check the possible combinations of ORFs that could have been selected to find the set of ORFs with the lowest average pairwise distances.

E.g. if we consider a single ortholog (ortholog_0), we have the four transcipts identified that the ORFs came from that were predicted to be orthologs, e.g. comp0, comp1, comp2, comp3. So that:
```python
transect_to_ORF_dict = {
    'comp0': ['comp0_m.0', 'comp0_m.1'],
    'comp1': ['comp1_m.0'],
    'comp2': ['comp2_m.0', 'comp2_m.1'],
    'comp3': ['comp3_m.0']
}
```
In this case the possible alignments that could have been selected were:
```python
list_of_possible_alignements = [
    ['comp0_m.0', 'comp1_m.0', 'comp2_m.0', 'comp3_m.0'],
    ['comp0_m.0', 'comp1_m.0', 'comp2_m.1', 'comp3_m.0'],
    ['comp0_m.1', 'comp1_m.0', 'comp2_m.0', 'comp3_m.0'],
    ['comp0_m.1', 'comp1_m.0', 'comp2_m.1', 'comp3_m.0'],   
]
```
For each of these alignments 2 way combinations irregardless of order were permutated and for each of these pairwise comparisons the sequence distance was calculated. For each alignment set, the average pairwise distance was then calculated and the set of sequences with the highest average pairwise alignment was selected as the best set of ORFs to work with.

For a sanity check through this process, I regularly inspected the alignments that were being out put as the 'best selections' to make sure that no further errors had occured previously in the one-to-one predictions. All alignments I checked looked great.

__Update (23/07/18):__ I am now working towards creating the dN/dS ratios using both CODEML and BUSTED and these will require us to work with the codon-based DNA rather than the AA. As such I will need to make some changes in the code below to make sure that I replace the DNA seqs in the CDS dataframes as well as the AA seqs.

Pseudo code:

In [ ]:
'''So now we know that there are a few doozies here that we need to take account of.
    1 - the comps were not unique and had sequence variations. These have been made unique in the
    longest_250 versions of the assemblies and so these are the files we should work with in terms of getting DNA
    2 - the comps are not unique across speceis, i.e. each species has a comp00001
    3 - after the above longest_250 processing we can essentially assume that comps are unique within species

    sooo.... some pseudo code to figure this out
    we should work in a dataframe for this
    read in the list of ortholog gene IDs for each species (the comps that make up the 19000) (this is the dataframe)
    then for each species, identify the gene IDs (comps) for which multiple ORFs were made
    go row by row in the dataframe and see if any of the comp IDs (specific to species) are with multiple ORFs
    These are our rows of interest, we need to work within these rows:

    for each comp for each speceis get a list of the orf aa sequences. turn these into a list of lists
    then use itertools.product to get all of the four orfs that could be aligned.
    then for each of these possible combinations do itertools.combinations(seqs, 2) and do
    pairwise distances and get the average score of the pairwise alignments
    we keep the set of four that got the best possible score
    '''

#### __Read in the csv files as pandas dataframes__
These will give us the compIDs and the actual aa seqs of the currently predicted ORF variants for each ortholog

__Update (23/07/18):__ We will also need to read in the cds dataframe as we will need to mofify this so that we can work with it when doing the dN/dS calculations.

In [ ]:
    # First get the ID of the transcript that is related to this Ortholog for each species
    base_dir = '/home/humebc/projects/parky'
    gene_id_df = pd.read_csv('/home/humebc/projects/parky/gene_id.csv', index_col=0)



    # We want to be able to compare how many of the alignments we fixed and how many were OK due to luck
    # To do this we will need a couple of counters but also the currently chosen ORFs
    aa_seq_df = pd.read_csv('/home/humebc/projects/parky/aa_seq.csv', index_col=0)

    # we will also need to update the cds_seq_df dataframe
    cds_seq_df = pd.read_csv('/home/humebc/projects/parky/cds.csv', index_col=0)

counters for keeping track of how many of the orthologs we had to fix

In [ ]:
multi_orf_counter = 0


#### Read in the files containing which ORFs were predicted for each of the transcripts for each species
Hold these in a 2D list, one list per species

__Update (23/07/18):__ We will also need to read in the cds version of this file so that we can get the correct DNA sequences and replace these in the cds dataframe.

In [ ]:
def convert_interleaved_to_sequencial_fasta_two(fasta_in):
    fasta_out = []

    for i in range(len(fasta_in)):

        if fasta_in[i].startswith('>'):
            if fasta_out:
                # if the fasta is not empty then this is not the first
                fasta_out.append(temp_seq_str)
            #else then this is the first sequence and there is no need to add the seq.
            temp_seq_str = ''
            fasta_out.append(fasta_in[i])
        else:
            temp_seq_str = temp_seq_str + fasta_in[i]
    #finally we need to add in the last sequence
    fasta_out.append(temp_seq_str)
    return fasta_out

In [ ]:
# read in the predicted orfs
# read in the orf aa files for each of the four species
try:
    comp_to_orfs_dict_holder_list = pickle.load( open('{}/comp_to_orfs_dict_holder_list.pickled'.format(base_dir), 'rb'))
    orf_to_aa_dict_holder_list = pickle.load( open('{}/orf_to_aa_dict_holder_list.pickled'.format(base_dir), 'rb'))
except:
    aa_orf_file_holder_list = []
    for spp in list(gene_id_df):
        with open('/home/baumgas/projects/done/7_John/assemblies_species/annotation/for-dnds/{}_ORFaa.fa'.format(spp),
                  'r') as f:
            aa_orf_file_holder_list.append(convert_interleaved_to_sequencial_fasta_two([line.rstrip() for line in f]))

    comp_to_orfs_dict_holder_list = [defaultdict(list) for spp in list(gene_id_df)]
    orf_to_aa_dict_holder_list = [dict() for spp in list(gene_id_df)]
    for i in range(len(list(gene_id_df))):
        for j in range(len(aa_orf_file_holder_list[i])):
            if aa_orf_file_holder_list[i][j].startswith('>'):
                comp_ID = aa_orf_file_holder_list[i][j].split()[8].split('_')[0]
                orf_ID = aa_orf_file_holder_list[i][j].split()[0][1:]
                comp_to_orfs_dict_holder_list[i][comp_ID].append(orf_ID)
                orf_to_aa_dict_holder_list[i][orf_ID] = aa_orf_file_holder_list[i][j + 1]
        # print out stats for the default dict to see if we agree with Chris at this point
        multi_orf_comps = sum([1 for k, v in comp_to_orfs_dict_holder_list[i].items() if len(v) > 1])
        print('{} comps with > 1 ORF in {}'.format(multi_orf_comps, list(gene_id_df)[i]))

    pickle.dump( comp_to_orfs_dict_holder_list, open('{}/comp_to_orfs_dict_holder_list.pickled'.format(base_dir), 'wb'))
    pickle.dump(orf_to_aa_dict_holder_list, open('{}/orf_to_aa_dict_holder_list.pickled'.format(base_dir), 'wb'))

#### Now do the same for the cds file

In [ ]:
# Here we will do the same as above but for the codon sequences.
# we will create both the com to orf dict and the orf to cds dict.
# in theory the comp to orf dict should be eactly the same for the cds file as it was for the aa file
# we should check this as a sanity check. If it is the same then we can ignore this second dict we made

try:
    comp_to_orfs_dict_holder_list_cds = pickle.load( open('{}/comp_to_orfs_dict_holder_list_cds.pickled'.format(base_dir), 'rb'))
    orf_to_cds_dict_holder_list = pickle.load( open('{}/orf_to_cds_dict_holder_list.pickled'.format(base_dir), 'rb'))
except:
    cds_orf_file_holder_list = []
    for spp in list(gene_id_df):
        with open('/home/baumgas/projects/done/7_John/assemblies_species/annotation/for-dnds/cds/{}_ORFcds.fa'.format(spp),
                  'r') as f:
            cds_orf_file_holder_list.append(convert_interleaved_to_sequencial_fasta_two([line.rstrip() for line in f]))

    comp_to_orfs_dict_holder_list_cds = [defaultdict(list) for spp in list(gene_id_df)]
    orf_to_cds_dict_holder_list = [dict() for spp in list(gene_id_df)]
    for i in range(len(list(gene_id_df))):
        for j in range(len(cds_orf_file_holder_list[i])):
            if cds_orf_file_holder_list[i][j].startswith('>'):
                comp_ID = cds_orf_file_holder_list[i][j].split()[8].split('_')[0]
                orf_ID = cds_orf_file_holder_list[i][j].split()[0][1:]
                comp_to_orfs_dict_holder_list_cds[i][comp_ID].append(orf_ID)
                orf_to_cds_dict_holder_list[i][orf_ID] = cds_orf_file_holder_list[i][j + 1]
        # print out stats for the default dict to see if we agree with Chris at this point
        multi_orf_comps = sum([1 for k, v in comp_to_orfs_dict_holder_list_cds[i].items() if len(v) > 1])
        print('{} comps with > 1 ORF in {}'.format(multi_orf_comps, list(gene_id_df)[i]))

    pickle.dump(comp_to_orfs_dict_holder_list_cds,
                open('{}/comp_to_orfs_dict_holder_list_cds.pickled'.format(base_dir), 'wb'))
    pickle.dump(orf_to_cds_dict_holder_list, open('{}/orf_to_cds_dict_holder_list.pickled'.format(base_dir), 'wb'))

#### Do sanity check to make sure that the ```compt_to_orfs_dict_holder_list_cds``` is the same as ```compt_to_orfs_dict_holder_list_cds```

In [ ]:
# SANITY CHECK: PASSED!
# let's look to make sure that both of the comp_to_orfs dicts are the same from reading in the aa and cds files.
for i in range(len(list(gene_id_df))):
    comp_dict_aa = comp_to_orfs_dict_holder_list[i]
    comp_dict_cds = comp_to_orfs_dict_holder_list_cds[i]
    for comp_key in comp_dict_aa.keys():
        if set(comp_dict_aa[comp_key]) != set(comp_dict_cds[comp_key]):
            sys.exit('The list of ORFs for comp {} do not match between the cds and aa dict'.format(comp_key))

#### Now make a list from this which is simply the comps that have multiple ORFs predicted for them

In [ ]:
# here we have the list of dictionaries from which we can get the comp IDs that have multiple ORFs
# perhaps we shoud convert them to lists now rather than on the fly
list_of_multi_orf_comps_per_spp = [[k for k, v in comp_to_orfs_dict_holder_list[i].items() if len(v) > 1] for i in range(len(list(gene_id_df)))]

#### Go through the df of compIDs for each ortholog and for each species look to see if the comp ID is found in the list we just created
If it is then this means that there at least one of the speceis has a transcript for which multiple ORFs were predicted. The set of ORFs associated with this ortholog will therefore need to be investigated.

__Update (23/07/18):__ Since we will be re-calculating the dN/dS ratios as well we will need to have a codon alignment to work with. We can use the aa s to make the tree but for the CODEML and the BUSTED analyses we would be working with codon alignments. Currently we convert the ORFs to their aa codes without keeping track of the ORF identifier. We then write out a 'fixed' aa dataframe. Instead of doing this we will need to keep track of the ORF ID so that we can also write out the corrected CDS sequence. To do this we have passed a tuple to the MP worker which contains both the aa_seq and the cds_seq. In the end we then fix the cds dataframe as well as the aa dataframe.

__Update (26/07/28):__ We were also having some problems with the fact that some of the cds sequences were not %3 compliant. We have incorporated screening into this code so that we drop any ortholog that has one or more ORFs that are not %3 compliant. (approximately 90 were not compliant, but after the alignment fixing here only about 8 were left).

We will multiprocess the checks of the problematic orthologs.
To do this we will create a list that will hold three items:
* A list of tuples. Each tuple containing four sets of both cds and aa sequences. The collection of tuples represent all of the possible alignments that we need to check
* The list of cds and aa sequenecs that are currently assigned to the ortholog
* The ortholog ID (simply an int)

#### create the list that will hold the MP info

In [ ]:
mp_list = []

In [ ]:
#  N.B. we have a problem here in that some of the CDS sequences are not 3%0.
# On first inspection it appears that 83 of the orthologs have CDS seqs that are not 3%0.
# 77 of these happen to be comps that have multi ORFs with 6 not having multi-ORFs
# I think it is very dangerous to be dealing with these partial ORFs as we don't know if they are partial in the
# 3' end or the 5' end.
# As such, I want to get rid of them using the following logic
# If one of an orthologs cds seqs is not 3%0 and doesn't have multi ORFs then we just drop this ortholog from
# the study.
# If above, but there is multi-ORFs then we will first find the best set of ORFs and then check to see if they
# contain any non 3%0 cds seqs. If they do then we will drop this ortholog too.

col_labels = list(gene_id_df)
ortholog_indices_to_drop = []
for index in gene_id_df.index.values.tolist():
    # within each row check to see if the comp is in its respective list

    row_to_check = False
    for i in range(len(col_labels)):
        if gene_id_df.loc[index, col_labels[i]] in list_of_multi_orf_comps_per_spp[i]:
            row_to_check = True
    # here we have checked through each of the comps in the row
    # if row_to_check == True then we need to work through each of the comps and see which ORF combinations
    # produce the best average pairwise distances

    if row_to_check:
        sys.stdout.write('\rChecking multi-ORF ortholog {} for MP'.format(index))
        multi_orf_counter += 1
        list_of_lists_of_possible_orfs = [comp_to_orfs_dict_holder_list[i][gene_id_df.loc[index, col_labels[i]]] for
                                          i in
                                          range(len(col_labels))]


        # convert the orf IDs associated to each of the possible ORFs to both aa_seqs and cds_seqs
        list_of_lists_of_possible_orfs_as_aa = [[] for spp in col_labels]
        list_of_lists_of_possible_orfs_as_cds = [[] for spp in col_labels]
        for k in range(len(list_of_lists_of_possible_orfs)):
            for orfID in list_of_lists_of_possible_orfs[k]:
                list_of_lists_of_possible_orfs_as_aa[k].append(orf_to_aa_dict_holder_list[k][orfID])
                list_of_lists_of_possible_orfs_as_cds[k].append(orf_to_cds_dict_holder_list[k][orfID])


        # hardcode it to a list so that we can get the index of each tuple below

        # to ensure maintainance of the direct link between the aa seq and the cds_seq it is maybe easiest
        # if instead of the unit of the tup being an aa sequence, it is a tuple itself which is (aa_seq, cds_seq)
        # this way we can know which of the cds_seqs were chosen based on the AAs/ORFIDs.
        # Create this list of list of tuples using the list_of_lists_of_possible_orfs and ..._as_aa lists
        list_of_lists_of_aa_to_cds_tups = [[] for spp in col_labels]
        for n in range(len(list_of_lists_of_possible_orfs)):
            for m in range(len(list_of_lists_of_possible_orfs[n])):
                aa_seq = list_of_lists_of_possible_orfs_as_aa[n][m]
                cds_seq = list_of_lists_of_possible_orfs_as_cds[n][m]
                list_of_lists_of_aa_to_cds_tups[n].append((aa_seq, cds_seq))

        # With the new change we should now be passing through a set of tuples that contain the aa_seq and the
        # ORF id to the MP
        alignment_tuples = [tup for tup in itertools.product(*list_of_lists_of_aa_to_cds_tups)]


        mp_list.append((alignment_tuples, aa_seq_df.loc[index].tolist(), cds_seq_df.loc[index].tolist(), index))
    else:
        # If the row does not need checking for multi-ORFs then we should still check to make sure that
        # the cds seqs for this ortholog are modulus 3 compliant else we should get rid of the ORF
        drop = False
        for spp in col_labels:
            if len(cds_seq_df.loc[index, spp])%3 != 0:
                drop = True
                break
        if drop:
            ortholog_indices_to_drop.append(index)

#### Setup the MP worker that will perform the pairwise comparisons

In [ ]:
def ORF_screening_worker(input_queue, rows_to_be_replaced_dict, indices_to_drop_list):
    #also verify that the cds sequences chosen are the same or different
    # to do this we will have to pass in the current cds seqs as well.

    for tup in iter(input_queue.get, 'STOP'):
        alignment_tuples, current_seqs_aa, current_seqs_cds, index = tup
        sys.stdout.write('\rAssessing multi-ORF ortholog: {} for best alignments'.format(index))

        # we should now have sets that are four aa sequences each paired with four cds seqs.
        # for each set, generate pairwise comparisons and calculate pairwise distances based on the aa_seqs
        # keep track of each distance and calculate average PW distances for each set_of_alignemtn_seqs
        # also associate which set of ORF_ids produced each average PW distance.
        # This list will keep track of this by storing tups that are (average PW distance)
        average_distance_list = []
        # it is important to note, and maintain the fact that the aa seqs and ORF_ids are in the species order
        # inside the alignment_tupes, i.e. first seq is always min
        for set_of_alignment_seqs in alignment_tuples:
            temp_pairwise_scores_list = []
            # here we are comparing two tuples that are (aa_seq, ORF_id)

            for seq_a, seq_b in itertools.combinations(set_of_alignment_seqs, 2):

                # here is a single PW distance calculation
                score = pairwise2.align.globalxx(seq_a[0], seq_b[0], score_only=True)
                temp_pairwise_scores_list.append(score)
            # now calculate average
            temp_average = sum(temp_pairwise_scores_list) / len(temp_pairwise_scores_list)
            average_distance_list.append(temp_average)

        # now we have a list of PW distance for each of the sets of sequences (virtual alignments)
        # we want to select the set that has the highest score

        index_of_best_set = average_distance_list.index(max(average_distance_list))
        # now check to see if these match those that were already chosen
        best_set_of_aa = [tup[0] for tup in alignment_tuples[index_of_best_set]]
        best_set_of_cds = [tup[1] for tup in alignment_tuples[index_of_best_set]]
        alignments_are_same = True

        for i in range(len(current_seqs_aa)):
            if current_seqs_aa[i] != best_set_of_aa[i]:
                alignments_are_same = False
                break

        if alignments_are_same:
            for i in range(len(current_seqs_cds)):
                if current_seqs_cds[i] != best_set_of_cds[i]:
                    alignments_are_same = False
                    break
        for i in range(len(current_seqs_cds)):
            if len(best_set_of_cds[i])%3 != 0:
                # if any of the new cds seqs break the 3%0 rule then we don't want to drop the ortholog
                indices_to_drop_list.append(index)
                # set alignments to true so that this ortholog will not be modified in the df s
                alignments_are_same = True
                break

        if not alignments_are_same:

            # we don't need to know the actual ORFs that have been chosen for each spp
            # only the aa codes so let's just output this result
            rows_to_be_replaced_dict[index] = [aa_orf_id_tup for aa_orf_id_tup in alignment_tuples[index_of_best_set]]

#### Setup and run the MP

In [ ]:
#here we need to make some changes. The dN/dS analysis that we are going to perform will be based on
# codon-based DNA sequences rather than amino acid seuences. As such we will need to have some way of
# correcting the cds dataframe as well as the AA dataframe.
num_proc = 20

#Queue that will hold the index of the rows that need to be checked
input_queue = Queue()

# populate input_queue
for tup in mp_list:
    input_queue.put(tup)

for i in range(num_proc):
    input_queue.put('STOP')

# Manager for a dict rows_to_be_replaced_dict that will hold the new (aa_seqs, ORF_id) for the fixed indices
manager = Manager()
rows_to_be_replaced_dict = manager.dict()
list_of_indices_to_drop = manager.list()

list_of_processes = []
for N in range(num_proc):
    p = Process(target=ORF_screening_worker, args=(input_queue, rows_to_be_replaced_dict, list_of_indices_to_drop))
    list_of_processes.append(p)
    p.start()

for p in list_of_processes:
    p.join()

At this point we have a dictionary that contains the ID of an ortholog as key and a list of the four aa seqs (and cds_seqs now) (one for each species) that need to be __replaced in the dataframe and then written out__.

In [ ]:
# add the multi proc list of indices to drop to the non-multi proc list
ortholog_indices_to_drop.extend(list(list_of_indices_to_drop))

# print out the orthologs that were non %3 compliant
with open('non_mod_3_compliant_orthologs.txt', 'w') as f:
    for line in ortholog_indices_to_drop:
        f.write('{}\n'.format(line))

# print out the orthologs that needed fixing
with open('multi_orf_orthologs_that_needed_fixing.txt', 'w') as f:
    for orth_id in rows_to_be_replaced_dict.keys():
        f.write('{}\n'.format(orth_id))

# now drop the indices from the dfs (all three)
aa_seq_df = aa_seq_df.drop(ortholog_indices_to_drop, axis=0)
cds_seq_df = cds_seq_df.drop(ortholog_indices_to_drop, axis=0)
gene_id_df = gene_id_df.drop(ortholog_indices_to_drop, axis=0)

fixed_counter = len(rows_to_be_replaced_dict.keys())
# now replace the dataframe values
# we can use a single parse on both the aa and cds
for index in aa_seq_df.index.values.tolist():
    if index in rows_to_be_replaced_dict.keys():
        # then this is a row that needs replcing with the new values
        aa_seq_df.loc[index] = [tup[0] for tup in rows_to_be_replaced_dict[index]]
        cds_seq_df.loc[index] = [tup[1] for tup in rows_to_be_replaced_dict[index]]


# at this point it only remains to write the dataframes out as csv
print('{} orthologs were checked due to multi-ORFs\n'
      '{} were fixed\n'
      '{} already contained the optimal choice of ORFs\n'.format
    (
    multi_orf_counter, fixed_counter, multi_orf_counter-fixed_counter
    )
)
aa_seq_df.to_csv('/home/humebc/projects/parky/aa_seq_multi_orf_orths__partial_cdsfixed.csv')
cds_seq_df.to_csv('/home/humebc/projects/parky/cds_seq_multi_orf_orths__partial_cdsfixed.csv')
gene_id_df.to_csv('/home/humebc/projects/parky/gene_id_multi_orf_orths_partial_cds_fixed.csv')

<hr />

<a id='creating_the_super_alignment_and_phylogeny'></a>
### Creating the super alignment and phylogeny
* Generate local alignment for each ortholog (MAFFT; cropped)
* Calculate best fit amino acid substitution matrix (using prottest)
* Concatenate the local alignements into super alignment, create q file and make ML tree (using raxml_HPC)

__Update (26/07/18):__
Since creating this tree I have moved on to re-doing the dN/dS analyses. This has meant putting the alignments through Guidance, to make sure that we are only relying on high confidence parts of the alignments. To run the CODEML and BUSTED components of this analysis I have used the alignments that have had low confidence regions removed. I have not used these slimmed down alignments for the tree making. Using the post-guidance alignments for the tree creation will make next to no difference for the end phylogeny. As such I have not yet reconstructed the tree using the post-guidance alignments.

Well aligned regions are of course critical for getting reliable dN/dS scores and as such I feel the guidance analysis is vital.

### Generate local alignment for each ortholog (MAFFT; cropped)

__Local functions__

In [1]:
def readDefinedFileToList(filename):
    temp_list = []
    with open(filename, mode='r') as reader:
        temp_list = [line.rstrip() for line in reader]
    return temp_list

def writeListToDestination(destination, listToWrite):
    #print('Writing list to ' + destination)
    try:
        os.makedirs(os.path.dirname(destination))
    except FileExistsError:
        pass

    with open(destination, mode='w') as writer:
        i = 0
        while i < len(listToWrite):
            if i != len(listToWrite)-1:
                writer.write(listToWrite[i] + '\n')
            elif i == len(listToWrite)-1:
                writer.write(listToWrite[i])
            i += 1

def convert_interleaved_to_sequencial_fasta_two(fasta_in):
    fasta_out = []

    for i in range(len(fasta_in)):

        if fasta_in[i].startswith('>'):
            if fasta_out:
                # if the fasta is not empty then this is not the first
                fasta_out.append(temp_seq_str)
            #else then this is the first sequence and there is no need to add the seq.
            temp_seq_str = ''
            fasta_out.append(fasta_in[i])
        else:
            temp_seq_str = temp_seq_str + fasta_in[i]
    #finally we need to add in the last sequence
    fasta_out.append(temp_seq_str)
    return fasta_out

#### Read in the aa and gene ID csvs

In [ ]:
# the amino acid sequences
aa_seq_array = pd.read_csv('/home/humebc/projects/parky/aa_seq_multi_orf_orths_fixed.csv', sep=',', lineterminator='\n', index_col=0, header=0)

# the gene IDs
gene_id_array = pd.read_csv('/home/humebc/projects/parky/gene_id_fixed.csv', sep=',', lineterminator='\n', index_col=0, header=0)


#### Do the alignments using multiprocessing to speed things up.
This worker will perform the alignment and also do the cropping. We will do the cropping by reading in an alignment into a dataframe and then drop the columns from the dataframe that contained gaps from both the beggining and the end, until we come to a column that doesn't contain a gap.

In [ ]:
def create_local_alignment_worker(input, output_dir, spp_list):
    # for each list that represents an ortholog
    for k_v_pair in iter(input.get, 'STOP'):

        # ortholog_id
        ortholog_id = k_v_pair[0]
        print('Processing {}'.format(ortholog_id))
        # ortholog_spp_seq_list
        ortholog_spp_seq_list = k_v_pair[1]

        # create the fasta
        fasta_file = []

        # for each species
        # add the name and aa_seq to the fasta_file
        for i in range(len(spp_list)):
            fasta_file.extend(['>{}_{}'.format(spp_list[i], ortholog_spp_seq_list[i][0]), ortholog_spp_seq_list[i][1]])

        # here we have the fasta_file populated

        # Write out the new fasta
        fasta_output_path = '{}/{}.fasta'.format(output_dir, ortholog_id)
        writeListToDestination(fasta_output_path, fasta_file)
        # now perform the alignment with MAFFT
        mafft = local["mafft-linsi"]
        in_file = fasta_output_path
        out_file = fasta_output_path.replace('.fasta', '_aligned.fasta')
        # now run mafft including the redirect
        (mafft[in_file] > out_file)()

        # at this point we have the aligned .fasta written to the output directory
        # at this point we need to trim the fasta.
        # I was going to use trimAl but this doesn't actually have an option to clean up the ends of alignments
        # instead, read in the alignment as a TwoD list to a pandas dataframe
        # then delete the begining and end columns that contain gap sites
        aligned_fasta_interleaved = readDefinedFileToList(out_file)
        aligned_fasta = convert_interleaved_to_sequencial_fasta_two(aligned_fasta_interleaved)
        array_list = []
        for i in range(1, len(aligned_fasta), 2):
                array_list.append(list(aligned_fasta[i]))

        # make into pandas dataframe
        alignment_df = pd.DataFrame(array_list)

        # go from either end deleting any columns that have a gap
        columns_to_drop = []
        for i in list(alignment_df):
            # if there is a gap in the column at the beginning
            if '-' in list(alignment_df[i]) or '*' in list(alignment_df[i]):
                columns_to_drop.append(i)
            else:
                break
        for i in reversed(list(alignment_df)):
            # if there is a gap in the column at the end
            if '-' in list(alignment_df[i]) or '*' in list(alignment_df[i]):
                columns_to_drop.append(i)
            else:
                break

        # get a list that is the columns indices that we want to keep
        col_to_keep = [col_index for col_index in list(alignment_df) if col_index not in columns_to_drop]

        # drop the gap columns
        alignment_df = alignment_df[col_to_keep]

        # here we have the pandas dataframe with the gap columns removed
        #convert back to a fasta and write out
        cropped_fasta = []
        alignment_index_labels = list(alignment_df.index)
        for i in range(len(alignment_index_labels)):
            seq_name = '>{}_{}'.format(spp_list[i], ortholog_spp_seq_list[i][0])
            aa_seq = ''.join(alignment_df.loc[alignment_index_labels[i]])
            cropped_fasta.extend([seq_name, aa_seq])

        # here we have the cropped and aligned fasta
        # write it out
        aligned_cropped_fasta_path = fasta_output_path.replace('.fasta', '_aligned_cropped.fasta')
        writeListToDestination(aligned_cropped_fasta_path, cropped_fasta)

        # here we should be done with the single alignment
        print('Local alignment for {} completed'.format(ortholog_id))

#### Create and populate the input Queue for the MP process. Then run.
The items in this list will be key value pairs from the dictionary that we will create below. The dictionary will simply be ortholog ID = key and list of aa seqs for that ortholog will be value.

In [ ]:
# making MP data_holder_list
tuple_holder_dict = {}
#for each ortholog
for row_index in aa_seq_array.index.values.tolist():
    print('Adding ortholog {} to MP info list'.format(row_index))
    ortholog_id_seq_list = []
    # for each species.
    for spp in list(gene_id_array):
        gene_id = gene_id_array[spp][row_index]
        aa_seq = aa_seq_array[spp][row_index]
        ortholog_id_seq_list.append((gene_id, aa_seq))
    tuple_holder_dict[row_index] = ortholog_id_seq_list

# creating the MP input queue
ortholog_input_queue = Queue()

# populate with one key value pair per ortholog
for key, value in tuple_holder_dict.items():
    print('Placing {} in MP queue'.format(key))
    ortholog_input_queue.put((key, value))

num_proc = 24

# put in the STOPs
for N in range(num_proc):
    ortholog_input_queue.put('STOP')

allProcesses = []

# directory to put the local alignments
output_dir = '/home/humebc/projects/parky/aa_tree_creation/local_alignments'

# the list of species for each ortholog
spp_list = [spp for spp in list(gene_id_array)]

# Then start the workers
for N in range(num_proc):
    p = Process(target=create_local_alignment_worker, args=(ortholog_input_queue, output_dir, spp_list))
    allProcesses.append(p)
    p.start()

for p in allProcesses:
    p.join()

# at this point we have the local alignments all written as fasta files to output_dir.
# Now it just remains to concatenate and then run the ML tree.

<hr />
### Calculate best fit amino acid substitution matrix (using prottest)
We will use prottest to calculate the model for each of the local alignments. We will MP this. This part of the program takes a considerable amount of time and I ran it over night to complete. You can run it using tmux. Once the models had been output I checked to see that none of the outputs had been corupted (and thus would cause problems further down the line) by simply checking the size of the output files. Most files were 33k in size. Files smaller than this were checked (about 20). These all had IO errors and had stopped prematurely. So, I ran the code again (it has a built in check to see if the output has already been produced; in which case it will not re-do). This time the check came up with no files less than 33k.

In [ ]:
def prottest_worker(input_queue):
    base_dir = '/home/humebc/projects/parky/aa_tree_creation/local_alignments'
    for file_name in iter(input_queue.get, 'STOP'):
        input_path = '{}/{}'.format(base_dir, file_name)
        output_path = input_path.replace('_aligned_cropped.fasta', '_prottest_result.out')
        if os.path.isfile(output_path):
            continue
        sys.stdout.write('\rRunning prottest: {}'.format(file_name))
        # perform the prottest
        prot_path = '/home/humebc/phylogeneticsSoftware/protest/prottest-3.4.2/prottest-3.4.2.jar'
        subprocess.run(['java', '-jar', prot_path, '-i', input_path, '-o', output_path, '-all-distributions', '-all']
                       , stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [ ]:
# we will find each of the local alignments and run put them into a list which we will MP
# for each item we will run prottest with a single thread and output a file
# in the concatenate local alignments file we will then create a q file that will
# designate the different partitions and which of the substitution models to use.

# get a list of all of the fasta names that we will want to concatenate
base_dir = '/home/humebc/projects/parky/aa_tree_creation/local_alignments'
list_of_files = [f for f in os.listdir(base_dir) if 'aligned_cropped.fasta' in f]


num_proc = 12

# Queue that will hold the index of the rows that need to be checked
input_queue = Queue()

# populate input_queue
for file_name in list_of_files:
    input_queue.put(file_name)

for i in range(num_proc):
    input_queue.put('STOP')

list_of_processes = []
for N in range(num_proc):
    p = Process(target=prottest_worker, args=(input_queue,))
    list_of_processes.append(p)
    p.start()

for p in list_of_processes:
    p.join()

return

<hr />
### Concatenate the local alignements into super alignment, create q file and make ML tree (using raxml_HPC)
To do this we read in all of the prot model output files and look to see which model was suggested for each of the local alignments. We make a default dictionary that holds key = model, value = list(the ortholog IDs that use that model). We then go model by model from this dict and within that ortholog ID by ortholog ID to concatenate all of the local alignments together. At the same time that we are doing the concatenation of the local alignments we are also producing the q_file on a model by model basis. The q file is a file that tells raxml how to partition the data and in our case which protein model should be used in conjunction with which partition. By grouping together the local alignments by the model they will use and therefore minimising the partitions we will hopefully be gaining significant advantages in comput time in the raxml stage.

#### Create the dictionary of model to ortholog IDs

In [ ]:
# The master alignment that we create should be partitioned according to the protein model used.
# I have generated all of the .out files which are the outputs from the prottest
# We should iter through these and create a dictionary that is a model type as key
# and then have a list of the orthologs of that model.
# then sort this by the length of the list
# then work our way through the local alignments in this order creating the alignment
# We will need to generate the p file as we go
# this should take the form
'''
JTT, gene1 = 1-500
WAGF, gene2 = 501-800
WAG, gene3 = 801-1000

'''

# get list of the .out prottest files
base_dir = '/home/humebc/projects/parky/aa_tree_creation/local_alignments'
list_of_prot_out_filenames = [f for f in os.listdir(base_dir) if 'prottest' in f]

# iter through the list of protfiles creating the dict relating model to ortholog
# we cannnot change the +G or +I for each partition. As such I will define according to the base model
model_to_orth_dict = defaultdict(list)
for i in range(len(list_of_prot_out_filenames)):
    model = ''
    file_name = list_of_prot_out_filenames[i]
    orth_num = int(file_name.split('_')[0])
    with open('{}/{}'.format(base_dir, file_name), 'r') as f:
        temp_file_list = [line.rstrip() for line in f]
    for j in range(300, len(temp_file_list), 1):
        if 'Best model according to BIC' in temp_file_list[j]:
            model = temp_file_list[j].split(':')[1].strip().replace('+G','').replace('+I','')
            break
    if model == '':
        sys.exit('Model line not found in {}'.format(orth_num))
    model_to_orth_dict[model].append(orth_num)

# #N.B. that we cannot have different gamma for different partitions
# # Also best advice is not to run +G and +I together.
# # As such we only need to extract the base model here i.e. WAG rather than WAG [+G|+I]
# for model in model_to_orth_dict

print('The 19k sequences are best represented by {} different aa models'.format(len(model_to_orth_dict.keys())))

#### Go model by model concatenating and making the q file

In [ ]:
# here we have the dict populated
# now sort the dict
sorted_model_list = sorted(model_to_orth_dict, key=lambda k: len(model_to_orth_dict[k]), reverse=True)

# now go model by model in the sorted_model_list to make the master alignment.

# not the most elegant way but I think I'll just create the mast fasta in memory
master_fasta = ['>min','', '>pmin', '', '>psyg', '', '>ppsyg', '']

# The q file will hold the information for the partitioning of the alignment for the raxml analysis
q_file = []
for model in sorted_model_list:
    q_file_start = len(master_fasta[1]) + 1
    sys.stdout.write('\rProcessing model {} sequences'.format(model))
    for orth_num in model_to_orth_dict[model]:
        file_name = str(orth_num) + '_aligned_cropped.fasta'
        with open('{}/{}'.format(base_dir, file_name), 'r') as f:
            temp_list_of_lines = [line.rstrip() for line in f]

        for i in range(1, len(temp_list_of_lines), 2):
            new_seq = master_fasta[i] + temp_list_of_lines[i]
            master_fasta[i] = new_seq
    q_file_finish = len(master_fasta[1])
    q_file.append('{}, gene{} = {}-{}'.format(
        model.upper(), sorted_model_list.index(model) + 1, q_file_start, q_file_finish))

# here we have the master fasta and the q file ready to be outputted

#### write out the master fasta and the q file

In [ ]:
# now write out the master fasta
master_fasta_output_path = '/home/humebc/projects/parky/aa_tree_creation/master.fasta'
with open(master_fasta_output_path, 'w') as f:
    for line in master_fasta:
        f.write('{}\n'.format(line))

# now write out the q file
q_file_output_path = '/home/humebc/projects/parky/aa_tree_creation/qfile.q'
with open(q_file_output_path, 'w') as f:
    for line in q_file:
        f.write('{}\n'.format(line))

#### Now it just remains to run the raxml.
There are a lot of options here to get it to run right
Breifly (to helpfully save time next time)
* -s = input file
* -q = the q file that defines the partitions
* -x = this switches on rapid bootstrapping and provides a random number to initiate
* -f a = Thi means that the summarised tree will also have the bootstrapped values put on it in the output
* -p = a seed required by raxml (random number)
* -# the number of bootstraps
* -n the base of the output files
* -w the output directory where the files will be written
* -T the threads used (processes)
* -m the model used (see comment in code)
* -B 0.03 auto determine the number of boostraps required to get stable support values

NB I tried to get the AVX2 version of raxml to work but the architechture of Symbiomics did not support it so we are running the AVX. Also please see the comment in the code on the -m flag.

In [ ]:
# now run raxml
#NB note that although we are specificing mdels for each partition, we still need to add the PROTGAMMAIWAG
# model argument to the -m flag. This is just a weird operation of raxml and is explained but hidden in the manual
# (search for 'If you want to do a partitioned analysis of concatenated'). Raxml will only extract the rate
# variation information from this and will ignore the model component e.g. WAG. FYI any model could be used
# doesn't have to be WAG.
raxml_path = '/home/humebc/phylogeneticsSoftware/raxml/standard-RAxML/raxmlHPC-PTHREADS-AVX'
subprocess.run([raxml_path, '-s', master_fasta_output_path, '-q', q_file_output_path,
                '-x', '183746', '-f', 'a', '-p', '83746273', '-#', '1000', '-T', '8', '-n', 'parkinson_out',
                '-m', 'PROTGAMMAWAG', '-w', '/home/humebc/projects/parky/aa_tree_creation'])

print('\nConstruction of master fasta complete:\n{}\n{}'.format(master_fasta_output_path, q_file_output_path))

__NB the outputs from the raxml are somewhat confusing!__
The support values are very hard to find and not displayed correctly using either treegraph2 or figtree. This online viewer did a good job though:
http://etetoolkit.org/treeview/?treeid=13316ffd89bd639d52886271ffab262b&algid=
The file to look for is the ```RAxML_bipartitionsBranchLabels.parkinson_out``` file. Look in the file and find the [XXX] for the percentage support. In our case 100%.

<hr></hr>

<a id='conducting_alignments_in_guidance'></a>
### Creating the guidance alignments (mafft) and dropping low confidence columns
In the code below we read in the cds fastas and we put them into guidance where we run a mafft alignment. We then look at the column scores for the aa alignments and from these we drop the columns of the cds alignments that have a score below 0.6. We then crop the alignments. These cds alignments are then what we'll use for the CODEML and the BUSTED analyses.

In [ ]:
def generate_local_alignments_for_each_ortholog_two():
    # So we were generating an alignment for the aa seqs using mafft and then cropping them
    # However, if we are going to use neighbour to do the aligning then that automatically produces us
    # a MAFFT alignment and allows us to screen for low scoring site that can additionally be translated
    # into removal of uncertain columns.
    # similar to how we were creating a directory for each of the aa_seqs we will do the same thing for
    # each of the orthologs.

    # the cds sequences
    cds_seq_df = pd.read_csv('/home/humebc/projects/parky/cds_seq_multi_orf_orths__partial_cdsfixed.csv', sep=',', lineterminator='\n',
                             index_col=0, header=0)

    # the list of species for each ortholog
    spp_list = [spp for spp in list(cds_seq_df)]


    #we will need a directory for each of the orthologs
    # in each directory we will have a cds fasta
    # to MP this we can simply give a list of the directories and get the ortholog ID from the directory name

    # for each ortholog, create directory and write in the fasta of the cds then pass the directory to the MP list

    MP_list = []


    for row_index in cds_seq_df.index.values.tolist():
        temp_fasta = []
        sys.stdout.write('\rGenerating fasta for ortholog {}'.format(row_index))

        # populate the fasta
        list_of_seq_names = []
        for spp in spp_list:
            temp_fasta.extend(['>{}_{}'.format(row_index, spp), cds_seq_df.loc[row_index, spp]])
            list_of_seq_names.append('{}_{}'.format(row_index, spp))
        # write out the fasta into the  directory
        base_dir = '/home/humebc/projects/parky/local_alignments/{0}/'.format(row_index)
        path_to_fasta = '/home/humebc/projects/parky/local_alignments/{0}/unaligned_cds_{0}.fasta'.format(row_index)
        os.makedirs(base_dir, exist_ok=True)
        with open(path_to_fasta, 'w') as f:
            for line in temp_fasta:
                f.write('{}\n'.format(line))

        # add the fasta full path to the MP list
        MP_list.append((path_to_fasta, list_of_seq_names))




    # creating the MP input queue
    ortholog_input_queue = Queue()

    # populate with one key value pair per ortholog
    for fasta_tup in MP_list:
        ortholog_input_queue.put(fasta_tup)

    num_proc = 24

    # put in the STOPs
    for N in range(num_proc):
        ortholog_input_queue.put('STOP')

    allProcesses = []


    # Then start the workers
    for N in range(num_proc):
        p = Process(target=guidance_worker, args=(ortholog_input_queue,))
        allProcesses.append(p)
        p.start()

    for p in allProcesses:
        p.join()

        # here we have the cds and the aa alignments cropped and written out
        # we can then use these as input into CODEML and the BUSTED programs
        # and to run the model calls for making the tree with.

def guidance_worker(input_queue):
    for fasta_tup in iter(input_queue.get, 'STOP'):
        fasta_file_full_path, seq_names = fasta_tup
        ortholog_id = fasta_file_full_path.split('/')[-2]
        sys.stdout.write('\rPerforming Guidance for {}'.format(ortholog_id))
        guidance_full_path = '/home/humebc/phylogeneticsSoftware/guidance2/guidance.v2.02/www/Guidance/guidance.pl'
        output_dir_full_path = '/'.join(fasta_file_full_path.split('/')[:-1])

        # check to see if guidance has already been completed

        # if os.path.isfile('{}/{}.cropped_aligned_cds.fasta'.format(output_dir_full_path, ortholog_id)):
        #     continue

        # subprocess.run(['perl', guidance_full_path, '--seqFile',
        #                 fasta_file_full_path, '--msaProgram', 'MAFFT', '--seqType',
        #                 'codon', '--outDir', output_dir_full_path, '--bootstraps', '10', '--outOrder', 'as_input',
        #                 '--colCutoff', '0.6', '--dataset', ortholog_id], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # # at this point we should have performed the guidance analysis
        # # we can now read in the cols that should be removed from the cds and aa alignments
        # cols_to_remove_cds = []
        # cds_cols_to_remove_file_path = 'Seqs.Orig_DNA.fas.FIXED.{}.MAFFT.Removed_Col'.format(ortholog_id)
        # with open(cds_cols_to_remove_file_path, 'r') as f:
        #     cds_col_file_list = [line.rstrip() for line in f]
        # for line in cds_col_file_list:
        #     cols_to_remove_cds.append(int(line.split()[2]))

        # at this point we should have performed the guidance analysis
        # we can now read in the scores for each aa residue and drop any alignment columns
        # that have only -na scores or contain a score < the cutoff which is 0.6
        # we will be working in sets of four because there are four sequences. Each column
        # we find to delte in this aa score matrix will represent three columns to be dropped
        # in the cds alignment.
        # also, bear in mind that the columns of the alignment are not 0 indexed but start at 1
        # we will have to take this into account when working out which columns of the alignments to drop

        cols_to_remove_aa = []
        aa_cols_score_file_path = '{}/{}.MAFFT.Guidance2_res_pair_res.PROT.scr'.format(output_dir_full_path, ortholog_id)

        with open(aa_cols_score_file_path, 'r') as f:
            aa_col_score_file_list = [line.rstrip() for line in f][1:]

        for i in range(int(len(aa_col_score_file_list)/4)):

            socres_set = []
            # get a list of the scores for each position
            # for j in range(i, i+4, 1):
            #     scores_set.append()
            # the range that represents i and the next three i s in the iteration without increasing i
            file_line_indices = [i*4 + k for k in range(4)]
            scores_set = [float(aa_col_score_file_list[j].split()[2]) if '-nan' not in aa_col_score_file_list[j] else '-nan' for j in file_line_indices]

            # now examine what is in the score sets

            drop = False
            nan_count = 0
            for score in scores_set:
                if score != '-nan':
                    if score < 0.6:
                        drop = True
                        break
                elif score == '-nan':
                    nan_count += 1
                else:
                    continue
            # when we come out ned to check if the nan_score == 4
            if nan_count == 4:
                drop = True


            # if drop = True then this is a column to drop
            if drop:
                cols_to_remove_aa.append(i)

        # here we have a 0 based list of the columns that need to be dropped from the aa alignment
        # convert this to a 0 based index of the columns that need to be dropped from the cds alignment
        cols_to_remove_cds = []
        for col_index in cols_to_remove_aa:
            cols_to_remove_cds.extend([col_index*3 + n for n in range(3)])

        # here we have the indices that need to be dropped for the cds and aa alignments
        # now read in the alignments as 2D lists, convert to pandas dataframe and perform the columns drops

        # aa first
        aa_alignment_file_path = '{}/{}.MAFFT.PROT.aln'.format(output_dir_full_path, ortholog_id)
        with open(aa_alignment_file_path, 'r') as f:
            aa_alignment_file_list = convert_interleaved_to_sequencial_fasta_two([line.rstrip() for line in f])
        aa_df = pd.DataFrame([list(line) for line in aa_alignment_file_list if not line.startswith('>')])

        # now drop the columns
        columns_to_keep_aa = [col for col in list(aa_df) if col not in cols_to_remove_aa]
        aa_df = aa_df[columns_to_keep_aa]

        # cds second
        cds_alignment_file_path = '{}/{}.MAFFT.aln'.format(output_dir_full_path, ortholog_id)
        with open(cds_alignment_file_path, 'r') as f:
            cds_alignment_file_list = convert_interleaved_to_sequencial_fasta_two([line.rstrip() for line in f])
        cds_df = pd.DataFrame([list(line) for line in cds_alignment_file_list if not line.startswith('>')])

        # now drop the columns
        columns_to_keep_cds = [col for col in list(cds_df) if col not in cols_to_remove_cds]
        cds_df = cds_df[columns_to_keep_cds]

        # here we have the cds and aa dfs that we can now do the cropping with and then finally write back out as
        # fasta files
        # go from either end deleting any columns that have a gap

        # aa first
        aa_df = crop_fasta_df(aligned_fasta_as_pandas_df_to_crop = aa_df)

        # cds second
        cds_df = crop_fasta_df(aligned_fasta_as_pandas_df_to_crop = cds_df)

        # now we just need to write out dfs
        aa_fasta_list = pandas_df_to_fasta(pd_df=aa_df, seq_names=seq_names)
        with open('{}/{}.cropped_aligned_aa.fasta'.format(output_dir_full_path, ortholog_id), 'w') as f:
            for line in aa_fasta_list:
                f.write('{}\n'.format(line))

        cds_fasta_list = pandas_df_to_fasta(pd_df=cds_df, seq_names=seq_names)
        with open('{}/{}.cropped_aligned_cds.fasta'.format(output_dir_full_path, ortholog_id), 'w') as f:
            for line in cds_fasta_list:
                f.write('{}\n'.format(line))

        # here we have the cds and the aa alignments cropped and written out
        # we can then use these as input into CODEML and the BUSTED programs

def pandas_df_to_fasta(pd_df, seq_names):
    temp_fasta = []
    for i in range(len(seq_names)):
        temp_fasta.extend(['>{}'.format(seq_names[i]), ''.join(list(pd_df.loc[i]))])
    return temp_fasta


def crop_fasta_df(aligned_fasta_as_pandas_df_to_crop):
    columns_to_drop = []
    for i in list(aligned_fasta_as_pandas_df_to_crop):
        # if there is a gap in the column at the beginning
        if '-' in list(aligned_fasta_as_pandas_df_to_crop[i]) or '*' in list(aligned_fasta_as_pandas_df_to_crop[i]):
            columns_to_drop.append(i)
        else:
            break
    for i in reversed(list(aligned_fasta_as_pandas_df_to_crop)):
        # if there is a gap in the column at the end
        if '-' in list(aligned_fasta_as_pandas_df_to_crop[i]) or '*' in list(aligned_fasta_as_pandas_df_to_crop[i]):
            columns_to_drop.append(i)
        else:
            break

    # get a list that is the columns indices that we want to keep
    col_to_keep = [col_index for col_index in list(aligned_fasta_as_pandas_df_to_crop) if col_index not in columns_to_drop]
    # drop the gap columns
    return aligned_fasta_as_pandas_df_to_crop[col_to_keep]

<a id='making_block_alignments_for_CODEML_analysis'></a>
### Creating phylip formated alignment blocks and running CODEML
The below code takes the guidance-created cds alignments and creates phylip formatted block alignments for the CODEML analysis. So the [documentation](http://nebc.nerc.ac.uk/bioinformatics/documentation/paml/doc/pamlDOC.pdf) for the CODEML input and analysis parameters are not so clear. Essentially, there are four ways to run the CODEML analysis, site specific, branch specific, site-branch specific and then there is what we want which is no site and no branch so just one dN/dS score per gene comparison. CODEML works by calling the execuatble (from its home directory; it relies on a relatively linked file) and by providing a control file that specifies which alignment to work with, which tree to work with, and where to output. It also specifies what sort of model we want to work with. For doing the pair-based analysis of the genes between species we can set the run mode to -2 (this is buried in the [docs](http://nebc.nerc.ac.uk/bioinformatics/documentation/paml/doc/pamlDOC.pdf)). You will see that we create the control file in the code below. You can run CODEML with a single gene at a time but a much faster way to do it is to provide it a phylip formated alignment file that contains multiple alignments (simply one alignment after the other). You can then tell CODEML in the control file how many alignments it should be expecting. Becuase there is no ability to MP analyses directly using the CODEML executable we split the ~19K genes into 20 blocks. Each block is an alignment file with 1000 alignemtns in it (except for the last one obviously which contained the remainder). We can then spawn a given number of processess that set off a CODEML analysis providing each of the blocks.

__Create the block alignments__

In [ ]:
def generate_block_phylip_alignments_for_CODEML():
    ''' The documentation for what format the files should be in for submitting to PAML/CODEML are not so
    great. But from some testing and looking through the examples that are available two things seem to be key
    1 - automatic pairwise comparisons of sequences can be performed using the runmode as -2
    2 - a blocked alignment format is the easiest way to test all of the orthologs at once but unlike
    in the documentation, (using the G marker) this is easiest done by simply having a phymil alignment format
    one after another in succestion in a sinlge file and then setting the ndata setting to how ever many
    sets of aligments you have (about 19K for us). I've run some tests and this all seems to be running well.
    Unfortunately, the only way to MP this seems to be physically splitting up the data and running multiple
    instance of the CODEML executable.
    So... with this in mind, I will split up the 19K or so sequences into phylip alignments of 1000 sequences.
    I will then make respective control files for each of these, put them in their own directory and then set
    an instance of the CODEML running for each of these.'''


    spp_list = ['min', 'pmin', 'psyg', 'ppsyg']
    # for each pairwise comparison of the species
    wkd = '/home/humebc/projects/parky/local_alignments'
    output_dir = '/home/humebc/projects/parky/guidance_analyses'
    list_of_guidance_dirs = []
    # set that will hold the ID of any orthologs that have dud alignments
    len_zero_dir_list = []
    # counter that we'll use to split into 20 roughly 1k sequence alignments
    counter = 0
    block_counter = 0

    # for each directory or ortholog
    list_of_dirs = list()
    for root, dirs, files in os.walk(wkd):
        list_of_dirs = dirs
        break
    for dir in list_of_dirs:

        sys.stdout.write('\rOrtholog: {}     {}/{}'.format(dir, counter, len(list_of_dirs)))

        # make a new alignment file for every 1000 individual alignments
        if counter % 1000 == 0:
            if block_counter != 0:

                # then we already have a block that needs writing
                seq_file_ctrl_file_tup = write_out_cntrl_and_seq_file(block_counter=block_counter, output_dir=output_dir,
                                             phylip_alignment=phylip_alignment, num_align=1000)
                list_of_guidance_dirs.append(seq_file_ctrl_file_tup)
            # once the old block is written out start a new one
            block_counter += 1
            os.makedirs('{}/block_{}'.format(output_dir, block_counter), exist_ok=True)
            print('\n\nStarting block {}'.format(block_counter))
            phylip_alignment = []

        # add single to block master alignment
        # if the fasta was empty then this will return False
        single_phylip = generate_phylip_from_fasta(dir, wkd)

        if single_phylip:
            phylip_alignment.extend(single_phylip)
            counter += 1
        else:
            # if the fasta was empty then log this and don't add anything to the counter
            len_zero_dir_list.append(dir)
    # write out a list of the poorl alignement orfs
    with open('post_guidance_0_len_cds_alignments_orthologs.txt', 'w') as f:
        for line in len_zero_dir_list:
            f.write('{}\n'.format(line))

    # now write out the final block of alignments
    seq_file_ctrl_file_tup = write_out_cntrl_and_seq_file(block_counter, output_dir, phylip_alignment, num_align=len(list_of_dirs)-len(len_zero_dir_list)-(1000*(block_counter-1)))
    list_of_guidance_dirs.append(seq_file_ctrl_file_tup)

    pickle.dump(list_of_guidance_dirs, open( '{}/list_of_guidance_dirs.pickle'.format(output_dir), "wb" ))

def generate_phylip_from_fasta(dir, wkd):
    temp_str = str()
    temp_list = list()

    with open('{}/{}/{}.cropped_aligned_cds.fasta'.format(wkd, dir, dir), 'r') as f:
        fasta_file = [line.rstrip() for line in f]

    if len(fasta_file[1]) == 0:
        # if the fasta is empty then we need to log this outside
        return False
    else:
        for line in fasta_file:
            if line.startswith('>'):
                temp_str = line[1:]
            else:
                temp_list.append('{}    {}'.format(temp_str, line))
        # finally put in the header file
        temp_list.insert(0, '\t{} {}'.format(len(temp_list), len(fasta_file[1])))

        return temp_list


def write_out_cntrl_and_seq_file(block_counter, output_dir, phylip_alignment, num_align):
    # write out the control file specific to this alignment
    ctrl_file_path = write_out_control_file(
        output_dir='{}/block_{}'.format(output_dir, block_counter),
        num_alignments=num_align,
        grp=block_counter)
    # write out the phylip file
    seq_file_path = '{}/block_{}/block_{}_cds.phylip'.format(output_dir, block_counter, block_counter)
    with open(seq_file_path, 'w') as f:
        for line in phylip_alignment:
            f.write('{}\n'.format(line))
    # write out the tree file
    tree_file = '(ppsyg:0.01524804457090833502,(min:0.00305561548082329418,pmin:0.00350296114601793013)' \
                ':0.03350192310501232812,psyg:0.01618135662493049715);'
    tree_file_path = '{}/block_{}/block_{}_tree.nwk'.format(output_dir, block_counter, block_counter)
    with open(tree_file_path, 'w') as f:
        f.write('{}\n'.format(tree_file))

    return (seq_file_path, ctrl_file_path, tree_file_path)

def write_out_control_file(output_dir, num_alignments, grp):
    seq_file_path = '{}/block_{}_cds.phylip'.format(output_dir, grp)
    out_file_path = '{}/block_{}_guidance_results.out'.format(output_dir, grp)
    ctrl_path     = '{}/block_{}_cds.ctrl'.format(output_dir, grp)
    tree_file_path = '{}/block_{}/block_{}_tree.nwk'.format(output_dir, grp, grp)
    ctrl_file = [
    'seqfile = {}'.format(seq_file_path),
    'treefile = {}'.format(tree_file_path),
    'outfile = {}'.format(out_file_path),
    'runmode = -2',
    'seqtype = 1',
    'CodonFreq = 2',
    'ndata = {}'.format(num_alignments),
    'clock = 0',
    'model = 0',
    'NSsites = 0',
    'icode = 0',
    'fix_omega = 0',
    'omega = .4',
    'cleandata = 0'
    ]

    with open(ctrl_path, 'w') as f:
        for line in ctrl_file:
            f.write('{}\n'.format(line))

    return ctrl_path

__Run the CODEML analyses__

In [ ]:
def run_CODEML_analyses():
    ''' We should read in the tuples that contain the seq files and cntrl file
    from the pickled files that were written out from generate_master_phlip_alignments_for_CODEML
    We should then start an MP list with each of these tuples in
    In the worker we should go to each directory and start an anlysis
    The 20000 sequences were broken into 20 chunks so we should start 20 subprocess instances and
    have a one CODEML analysis run in each'''

    tup_of_dirs_list = pickle.load( open( '/home/humebc/projects/parky/guidance_analyses/list_of_guidance_dirs.pickle', "rb" ))

    input_queue = Queue()

    for tup in tup_of_dirs_list:
        input_queue.put(tup)

    num_proc = 1

    for i in range(num_proc):
        input_queue.put('STOP')

    list_of_processes = []
    for N in range(num_proc):
        p = Process(target=CODEML_run_worker, args=(input_queue,))
        list_of_processes.append(p)
        p.start()

    for p in list_of_processes:
        p.join()


def CODEML_run_worker(input_queue):
    CODEML_path = '/home/humebc/phylogeneticsSoftware/paml/paml4.9h/bin/codeml'
    for dir_tup in iter(input_queue.get, 'STOP'):
        seq_file_path, ctrl_file_path, tree_file_path = dir_tup

        wkd = os.path.dirname(seq_file_path)

        os.chdir(wkd)

        subprocess.run([CODEML_path, ctrl_file_path])

__Collect the results of the CODEML analysis into one dataframe__

In [ ]:
def summarise_CODEML_pairwise_analyses():
    ''' Here we will go through each of the block analyses and take out the pairwise dn/ds analyses and put
    them into a single pandas dataframe. The ortholog will the index (which we will evenutally sort) and the
    columns will be each of the pairwise comparisons.'''

    block_analysis_base_dir = '/home/humebc/projects/parky/guidance_analyses'

    list_of_dirs = list()
    for root, dirs, files in os.walk(block_analysis_base_dir):
        list_of_dirs = dirs
        break

    # we will create a index system for the columns so that each pairwise difference will be in a specific column
    # of the df that we are going to create
    # col1 = min_pmin
    # col2 = min_psyg
    # col3 = min_ppsyg
    # col4 = pmin_psyg
    # col5 = pmin_ppsyg
    # col6 = psyg_ppsyg

    comparisons = ['min_pmin', 'min_psyg', 'min_ppsyg', 'pmin_psyg', 'pmin_ppsyg', 'psyg_ppsyg']
    df = pd.DataFrame(columns=comparisons)
    count = 0
    # the list that we will hold the info for a single row in
    dict_of_dnns_values = {}
    for block_dir in list_of_dirs:
        sys.stdout.write('\n\nProcessing block {}\n\n'.format(block_dir))
        # read in the file for this block and populate the df with the dN/dS videos
        with open('{0}/{1}/{1}_guidance_results.out'.format(block_analysis_base_dir, block_dir), 'r') as f:
            out_file = [line.rstrip() for line in f]

        # go line by line through the output file picking up the dnds values and putting them into the df
        for i in range(len(out_file)):
            if 'Data set' in out_file[i]:
                sys.stdout.write('\rProcessing {}'.format(out_file[i]))
            # see if the line in question contains dnds values
            if 'dN/dS=' in out_file[i]:
                # check to see if we have populated a row worth of dnds values
                # if so, populate the df and then start a new row list to collect values in
                if count % 6 == 0 and count != 0:
                    # then we should have a set of dnds values that we can append to the df
                    df.loc[int(ortholog_id)] = [dict_of_dnns_values[pair] for pair in comparisons]
                    dict_of_dnns_values = {}

                # when we are here we are either adding one more value to an already exisiting set
                # or we are starting a brand new set

                # get the dn/ds value and the pair that we are working with
                dn_ds_value = float(out_file[i].split()[7])
                orth_and_pair_info_line = out_file[i-4]
                pair_one = orth_and_pair_info_line.split()[1]
                pair_two = orth_and_pair_info_line.split()[4]
                ortholog_id = pair_one.split('_')[0][1:]
                spp_one = pair_one.split('_')[1][:-1]
                spp_two = pair_two.split('_')[1][:-1]

                if '{}_{}'.format(spp_one, spp_two) in comparisons:
                    dict_of_dnns_values['{}_{}'.format(spp_one, spp_two)] = dn_ds_value
                else:
                    dict_of_dnns_values['{}_{}'.format(spp_two, spp_one)] = dn_ds_value

                count += 1
    # finally add the last set of dn/ds data to the df
    df.loc[int(ortholog_id)] = [dict_of_dnns_values[pair] for pair in comparisons]
    pickle.dump( df, open('{}/CODEML_dnds_pairwise_pandas_df.pickle'.format(block_analysis_base_dir), 'wb'))

    apples = 'asdf'

__Create a stats output for the CODEML analysis__

In [ ]:
def generate_summary_stats_for_CODEML_analysis():
    block_analysis_base_dir = '/home/humebc/projects/parky/guidance_analyses'

    df = pickle.load( open('{}/CODEML_dnds_pairwise_pandas_df.pickle'.format(block_analysis_base_dir), 'rb'))
    df.sort_index(inplace=True)
    df.to_csv('/home/humebc/projects/parky/guidance_analyses/CODEML_dN_dS.csv')
    comparisons = ['min_pmin', 'min_psyg', 'min_ppsyg', 'pmin_psyg', 'pmin_ppsyg', 'psyg_ppsyg']

    
    count_dict = defaultdict(list)

    
    pairwise_dict = defaultdict(list)

    total_count = 0
    for index in df.index.values.tolist():
        sys.stdout.write('\rProcessing index: {}'.format(index))
        count = 0
        for comp in comparisons:
            if df.loc[index, comp] > 1 and df.loc[index, comp] != float(99):
                count +=1
                total_count += 1
                pairwise_dict[comp].append(index)
        count_dict[count].append(index)

    f = open('{}/CODEML_dN_dS_results.txt'.format(block_analysis_base_dir), 'w')

    print('PAIRWISE COUNTS:')
    f.write('PAIRWISE COUNTS:\n')
    for comp in comparisons:
        print('{}: {}'.format(comp, len(pairwise_dict[comp])))
        f.write('{}: {}\n'.format(comp, len(pairwise_dict[comp])))
    print('\n\nCOUNTS')
    f.write('\n\nCOUNTS\n')
    count = 0
    for i in [1,2,3,4, 5, 6]:
        print('Orthologs with {} ORF dN/dS > 1: {} '.format(i, len(count_dict[i])))
        f.write('Orthologs with {} ORF dN/dS > 1: {} \n'.format(i, len(count_dict[i])))
        count += len(count_dict[i])
    print('\n\nNumber of orthologs with at least one >1 dN/dS scoring ORF: {}'.format(count))
    f.write('\n\nNumber of orthologs with at least one >1 dN/dS scoring ORF: {}\n'.format(count))
    f.close()

    

<a id='creating_busted_input_files_and_running_analysis'></a>
### Creating BUSTED input files and running analysis
BUSTED analyses can be run through an online GUI called datamonkey. Or it can be run on the command line. However, running it on the command line is annoying because it is interactive. As such, to be able to run it from python on all of the 19K genes (one analysis per gene) we had to create 'answer_files' that contained the answers to the interactive questions that the busted executable wanted to run. You will see that these are generated in the code below. To do this we called the BUSTED executable with subprocess.run but we modified the input argument passing in the actual answers and also the stdout so that it was piped straight into an output file.

__Create a directory per gene and put the tree and fasta in. Then run analysis MP__

In [ ]:
def create_directories_and_run_busted():
    ''' The BUSTED annoyingly has an interactive interface. It does accept special batch files, but I really
    don't want to have to invest the time in learning how to use those just for this analysis.
    Instead I will simply use a text file that had the predetermined answers to the interactive prompts in
    order to start analyses.
    To conduct the analyses we will have to go ortholog by ortholog, pulling out the fasta alignment
    and then writing this to a new directory were we will also write the tree. Then we simply write
    the answer file and run the command.

    We should definitely MP this. To do this we should read in the directories
    and put these in as the MP list arguments. We should then make all of the files etc and run the analyses
    within these threads.

    we need to remember to check that alignments aren't empty.'''

    local_dirs_dir = '/home/humebc/projects/parky/local_alignments'
    list_of_dirs = list()
    for root, dirs, files in os.walk(local_dirs_dir):
        list_of_dirs = dirs
        break

    input_queue = Queue()

    for dir in list_of_dirs:
        input_queue.put(dir)

    num_proc = 40

    for i in range(num_proc):
        input_queue.put('STOP')

    list_of_processes = []
    for N in range(num_proc):
        p = Process(target=BUSTED_MP_worker, args=(input_queue,))
        list_of_processes.append(p)
        p.start()

    for p in list_of_processes:
        p.join()

    apples = 'asdf'

def BUSTED_MP_worker(input_queue):
    busted_analyses_dir = '/home/humebc/projects/parky/busted_analyses'
    local_dirs_dir = '/home/humebc/projects/parky/local_alignments'


    # The HYPHYMP executable relies on some sort of relative file but I can't find out which one
    # as such we will have to change dir to the HYPHY_idr in order to be able to invoke the program.
    HYPHYMP_path = '/home/humebc/phylogeneticsSoftware/hyphy/hyphy-2.3.13/HYPHYMP'
    HYPHY_dir = '/home/humebc/phylogeneticsSoftware/hyphy/hyphy-2.3.13'
    for ortholog_id in iter(input_queue.get, 'STOP'):

        # First check to see if the busted processing has already been done for this ortholog
        # if it has then skip onto the next one.
        if os.path.isfile('{0}/{1}/{1}_qced_cds_aligned.fasta.BUSTED.json'.format(busted_analyses_dir, ortholog_id)):
            continue

        # First check to see that the alignment is good
        orig_align_path = '{0}/{1}/{1}.cropped_aligned_cds.fasta'.format(local_dirs_dir, ortholog_id)
        with open(orig_align_path, 'r') as f:
            orig_fasta = [line.rstrip() for line in f]

        if len(orig_fasta[1]) < 2:
            # then the fasta is bad and we should move onto the next directory and ignore this one.
            continue


        sys.stdout.write('\rRunning BUSTED analysis on ortholog: {}'.format(ortholog_id))
        tree_file = '({0}_ppsyg:0.01524804457090833502,({0}_min:0.00305561548082329418,{0}_pmin:0.00350296114601793013)' \
                    ':0.03350192310501232812,{0}_psyg:0.01618135662493049715);'.format(ortholog_id)

        wkd = '{}/{}'.format(busted_analyses_dir, ortholog_id)
        os.makedirs(wkd, exist_ok=True)

        # write out the tree to the busted analysis directory in question
        tree_path = '{}/{}_tree.nwk'.format(wkd, ortholog_id)
        with open(tree_path, 'w') as f:
            f.write('{}\n'.format(tree_file))

        # copy the cds fasta alignment from the local directory to this directory

        new_align_path = '{}/{}_qced_cds_aligned.fasta'.format(wkd, ortholog_id)
        shutil.copyfile(orig_align_path, new_align_path)

        # here we now have the alignment and the tree file in the directory we want.

        # now write out the text file that we will use to put in our answeres to the interactive prompts
        answers_script = ['1', '5', '1', new_align_path, tree_path, '1']
        answers_script_path = '{}/{}_answers'.format(wkd, ortholog_id)
        with open(answers_script_path, 'w') as f:
            for line in answers_script:
                f.write('{}\n'.format(line))

        # change dir to the hyphy dir
        output_file_path = '{}/{}_results.out'.format(wkd, ortholog_id)
        os.chdir(HYPHY_dir)
        # HYPHYMP_cmd = './HYPHYMP < {} > {}'.format(answers_script_path, output_file_path)
        # p = subprocess.Popen(HYPHYMP_cmd, shell=True)
        # os.waitpid(p.pid, 0)
        # subprocess.run(['./HYPHYMP', '<', answers_script_path, '>',  output_file_path])
        with open(output_file_path, 'w') as output:
            subprocess.run([HYPHYMP_path], stdout=output, input='\n'.join(answers_script) + '\n', encoding='ascii')

        apples = 'adsf'


    return

__Collect the busted analyses into a dataframe__

In [ ]:
def summarise_BUSTED_analyses():
    busted_base_dir = '/home/humebc/projects/parky/busted_analyses'

    list_of_dirs = list()
    for root, dirs, files in os.walk(busted_base_dir):
        list_of_dirs = dirs
        break

    manager = Manager()
    managed_list = manager.list()

    input_queue = Queue()

    for dir in list_of_dirs:
        input_queue.put(dir)

    num_proc = 40

    for i in range(num_proc):
        input_queue.put('STOP')

    list_of_processes = []
    for N in range(num_proc):
        p = Process(target=collect_busted_MP_worker, args=(input_queue,managed_list))
        list_of_processes.append(p)
        p.start()

    for p in list_of_processes:
        p.join()




    df = pd.DataFrame([tup[1] for tup in list(managed_list)], columns=['p_value_div_selct'], index=[tup[0] for tup in list(managed_list)])
    # for ortholog_id, p_value_float in list(managed_list):
    #     df.loc[int(ortholog_id)] = p_value_float
    df.sort_index(inplace=True)
    pickle.dump(df, open('{}/BUSTED_p_value_results.pickle'.format(busted_base_dir), 'wb'))
    df.to_csv('{}/BUSTED_p_value_results.csv'.format(busted_base_dir))
    apples = 'asdf'

def collect_busted_MP_worker(input_queue, managed_list):
    busted_base_dir = '/home/humebc/projects/parky/busted_analyses'
    for directory in iter(input_queue.get, 'STOP'):
        sys.stdout.write('\rCollecting ortholog for {}'.format(directory))
        with open('{0}/{1}/{1}_results.out'.format(busted_base_dir, directory)) as f:
            output_file = [line.rstrip() for line in f]

        for i in range(len(output_file)):
            if output_file[i] == 'MATRIX':
                ortholog_id = int(output_file[i + 1].split('_')[0].lstrip()[1:])
            if 'Likelihood ratio test for episodic' in output_file[i]:
                p_value_float = float(output_file[i].split()[-1][:-3])

        managed_list.append((ortholog_id, p_value_float))

__Out put a stats file from the df__

In [ ]:
def summarise_busted_results_stats():
    busted_base_dir = '/home/humebc/projects/parky/busted_analyses'
    df = pickle.load( open('{}/BUSTED_p_value_results.pickle'.format(busted_base_dir), 'rb'))

    sig_at_dict = defaultdict(list)

    for index in df.index.values.tolist():
        p_val = df.loc[index, 'p_value_div_selct']
        if p_val < 0.05:
            sig_at_dict[0.05].append(index)
        if p_val < 0.01:
            sig_at_dict[0.01].append(index)
        if p_val < 0.001:
            sig_at_dict[0.001].append(index)

    f = open('{}/summary_stats_for_busted_p_vals.txt'.format(busted_base_dir), 'w')

    print('Number of orthologs found to have significant diversifying selection at:\n\n')
    print('p < 0.05 = {}'.format(len(sig_at_dict[0.05])))
    print('p < 0.01 = {}'.format(len(sig_at_dict[0.01])))
    print('p < 0.001 = {}'.format(len(sig_at_dict[0.001])))
    f.write('Number of orthologs found to have significant diversifying selection at:\n\n')
    f.write('p < 0.05 = {}\n'.format(len(sig_at_dict[0.05])))
    f.write('p < 0.01 = {}\n'.format(len(sig_at_dict[0.01])))
    f.write('p < 0.001 = {}\n'.format(len(sig_at_dict[0.001])))
    f.close()